# Постановка задачи

На сегодняшний день машинное обучение решает абсолютно разные задачи практически во всех областях нашей жизни. Платформа DataDriven организовывает контесты по машинному обучению для решения реальных задач общественных и некоммерческих организаций по всему миру, применяя результаты победителей. Данный проект призван решить проблемы водоснабжения в стране Танзания в Восточной Африке. Вода в Танзании добывается с помощью водокачек, и они часто выходят из строя, оставляя поселения без воды. Задача участников - по параметрам и статусу некоторого множества водокачек определить возможность поломки и необходимость обслуживания других водокачек. 

В данной задаче мы не только сделаем предсказания о состоянии водокачек, но и оценим, как качество данных влияет на предсказание.

Ссылка на контест: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/



# Описание метода

Задача будет решаться с помощью градиентного бустинга - будем использовать одноимённый метод GradientBoosting из пакета sklearn.

## Практическая суть задачи

Пусть у нас существует некоторый набор пар {(𝑥𝑖,𝑦𝑖)}, i=1..n, где x - это признак, y - целевая переменная. Задача сводится к тому, чтобы восстановить функцию y=F(x) - функцию, минимизирующую функцию потерь L(x, F(x))

## Градиентный бустинг

Метод градиентного бустинга позволяет построить итоговую модель F(x) как взвешенную функцию слабых моделей Fm(x), m = 1..M. К каждой предыдущей сумме Fm-1(x) добавляется частная производная от функции L(x, Fm(x)) по Fm(x) со знаком минус - для минимизации потерь. Назовём эту производную hm(x). Необходимо также, чтобы функция L(x, Fm(x)) была дифференцируема в достаточно большой области. 

In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier

Для работы будем использовать библиотеку Pandas. Выведем следующие данные:
TrainingX - множество признаков - набор данных о характеристиках водокачек
TrainingY - множество целевых переменных - набор данных о состоянии водокачек
TestX - множество признаков - набор данных о характеристиках водокачек, по которым требуется совершить предсказание.

In [4]:
TrainingX = pd.read_csv('./contest/trainingsetvalues.csv', sep=',')
TrainingY = pd.read_csv('./contest/trainingsetlabels.csv', sep=',')
TestX = pd.read_csv('./contest/testsetvalues.csv', sep=',')

Посмотрим, что представляют из себя данные о характеристиках водокачек:

In [5]:
print(type(TrainingX))

<class 'pandas.core.frame.DataFrame'>


In [6]:
TrainingX.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


# Пояснения к данным

Перечень столбцов с расшифровкой:
amount_tsh - Общий статический напор (количество воды, доступное для водокачки)
date_recorded - Дата ввода данных
funder - Кто финансировал колодец
gps_height - Глубина скважины
installer - Организация, вырывшая скважину
longitude - GPS-координата
latitude - GPS-координата
wpt_name - Название водоема, если он есть
num_private -
basin - Географическое расположение водоёма
subvillage - Географическое положение населённого пункта
region - Географическое положение региона
region_code - Код региона
district_code - Код района
lga - Географическое положение
ward - Географическое положение
population - Население вокруг скважины
public_meeting - Верно/Ложно
recorded_by - Групповой ввод этой строки данных
scheme_management - Кто управляет водокачкой
scheme_name - Кто управляет водокачкой
permit - Разрешен ли водозабор
construction_year - Год постройки водозабора
extraction_type - Тип добычи воды, которую использует водонапорная точка
extraction_type_group - Подвид добычи воды, которую использует водонапорная точка
extraction_type_class - Класс добычи воды, которую использует водонапорная точка
management - Как управляется водокачка
management_group - Как управляется водокачка
payment - Сколько стоит вода
payment_type - Сколько стоит вода
water_quality - Качество воды
quality_group - Качество воды
quantity - Количество воды
quantity_group - Количество воды
source - Источник воды
source_type - Источник воды
source_class - Источник воды
waterpoint_type - Вид водоема
waterpoint_type_group - Вид водоема


# Распределение данных по параметрам строковых типов
Чтобы включить в модель такие признаки, необходимо, чтобы было достаточно данных по каждому из них, и чтобы данные были достаточно равномерно распределены по ним. Для начала посмотрим, какие типы столбцов и в каком количестве у нас имеются:

In [12]:
TrainingX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Как мы видим,  строковых значений в таблице 30, целочисленных - 7, вещественных - 3. Посмотрим на таблицу, содержащую только строковые типы данных:

In [16]:
TrainingX.select_dtypes(include=['object'])

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,2013-03-06,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,NaN,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,2011-07-13,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,2013-05-03,Germany Republi,CES,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi,True,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,2011-05-07,Cefa-njombe,Cefa,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo,True,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,2011-04-11,NaN,NaN,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala,True,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,2011-03-08,Malec,Musa,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu,True,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


## Практическая оценка качества данных
Для начала попробуем оценить качество данных, построив модель напрямую. Оценим точность предсказания, разделив данные на две части: на одной обучим модель, по второй оценим точность.

Делим данные на два датафрейма:

In [196]:
TrainX1 = TrainingX[:29700]
TrainY1 = TrainingY[:29700]
TrainX2 = TrainingX[29700:]

Создаём объект модели:

In [ ]:
model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=5,
        #subsample=0.5,
        max_features="auto"
        # random_state=seed
    )

Включаем в выборку все столбцы, приводимые к float напрямую, и приводим типы данных:

In [213]:
TrainX1.dtypes == object


id                       False
amount_tsh               False
date_recorded             True
funder                    True
gps_height               False
installer                 True
longitude                False
latitude                 False
wpt_name                  True
num_private              False
basin                     True
subvillage                True
region                    True
region_code              False
district_code            False
lga                       True
ward                      True
population               False
public_meeting            True
recorded_by               True
scheme_management         True
scheme_name               True
permit                    True
construction_year        False
extraction_type           True
extraction_type_group     True
extraction_type_class     True
management                True
management_group          True
payment                   True
payment_type              True
water_quality             True
quality_

id                       False
amount_tsh               False
date_recorded            False
funder                   False
gps_height               False
installer                False
longitude                False
latitude                 False
wpt_name                 False
num_private              False
basin                    False
subvillage               False
region                   False
region_code              False
district_code            False
lga                      False
ward                     False
population               False
public_meeting           False
recorded_by              False
scheme_management        False
scheme_name              False
permit                   False
construction_year        False
extraction_type          False
extraction_type_group    False
extraction_type_class    False
management               False
management_group         False
payment                  False
payment_type             False
water_quality            False
quality_

In [207]:
ctrainX = TrainX1[['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']]
#ctrainX = ctrainX.values()
ctrainX = ctrainX.astype(float)
ctrain

ValueError: could not convert string to float: 'Roman'

## Оптимизация данных
Для начала нам надо определить, есть ли записи об одних и тех же водокачках и насколько различаются данные по ним. Уникальными значениями для каждой из водокачек являются их GPS-координаты. Оценим, как много в таблице уникальных водокачек по координатам:

In [49]:
len(TrainingX['latitude'].unique())

57517

In [46]:
len(TrainingX['longitude'].unique())

57516

In [155]:
Train = TrainingX.loc[TrainingX['longitude'] != 0]
#TrainingX.groupby('latitude')['longitude']

Train[Train.duplicated(subset='longitude',keep=False)][Train.duplicated(subset='latitude',keep=False)]




c:\python3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
1317,62247,2000.0,2011-07-19,P,1262,P,33.015597,-2.487085,Kwa Chongo Kideshe,0,...,monthly,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
1901,38641,100.0,2013-03-16,NaN,65,NaN,39.090880,-6.981830,Kwa Bwana Shamba Wa Manispaa,0,...,per bucket,salty abandoned,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
2287,42469,0.0,2011-03-24,Nethalan,583,RWE,37.339811,-7.065373,Ofisini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
2401,58328,0.0,2011-07-20,H,0,H,32.988560,-2.489378,Kwa Kaombwe,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
2457,73178,0.0,2011-07-20,L,1195,L,32.956523,-2.494353,Kwa Julias Mabula,0,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,other,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57390,60604,50.0,2013-03-15,NaN,62,NaN,39.096499,-6.980216,Kwa Semkwa,0,...,per bucket,salty abandoned,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
57423,38295,0.0,2011-03-27,W.D & I.,557,RWE,37.540901,-6.959749,Kwa Nasoro Kinenge,0,...,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
57824,29698,1000.0,2011-03-24,Nethalan,621,RWE,37.375717,-7.056372,Kwa Yusta,0,...,monthly,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
58190,58212,0.0,2011-07-15,H,0,H,32.978062,-2.515321,Kwa Buswelu B,0,...,monthly,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


Как мы видим, нашлось 138 водокачек с одинаковыми ненулевыми координатами. Многие из них были профинансированы или построены одними и теми же лицами, что может указывать на то, что это данные об одних и тех же водокачках. Проверим их на совпадение годов ввода в эксплуатацию:

In [156]:
TrainYEAR = Train[Train.duplicated(subset='longitude',keep=False)][Train.duplicated(subset='latitude',keep=False)]
TrainYEAR[TrainYEAR.duplicated(subset='construction_year', keep = False)]

c:\python3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
1901,38641,100.0,2013-03-16,NaN,65,NaN,39.090880,-6.981830,Kwa Bwana Shamba Wa Manispaa,0,...,per bucket,salty abandoned,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
2287,42469,0.0,2011-03-24,Nethalan,583,RWE,37.339811,-7.065373,Ofisini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
2401,58328,0.0,2011-07-20,H,0,H,32.988560,-2.489378,Kwa Kaombwe,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
4125,43485,0.0,2011-07-19,P,0,P,33.015597,-2.487085,Kwa Chongo Kideshe,0,...,monthly,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
4488,7415,0.0,2011-07-15,H,1230,H,32.977191,-2.516619,Kwa Buswelu B,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57390,60604,50.0,2013-03-15,NaN,62,NaN,39.096499,-6.980216,Kwa Semkwa,0,...,per bucket,salty abandoned,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
57423,38295,0.0,2011-03-27,W.D & I.,557,RWE,37.540901,-6.959749,Kwa Nasoro Kinenge,0,...,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
57824,29698,1000.0,2011-03-24,Nethalan,621,RWE,37.375717,-7.056372,Kwa Yusta,0,...,monthly,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
58190,58212,0.0,2011-07-15,H,0,H,32.978062,-2.515321,Kwa Buswelu B,0,...,monthly,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


Посмотрим на повторяющиеся данные:

In [157]:
TrainYEAR.loc[abs(TrainYEAR['longitude'] - 37.3) < 0.1]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
2287,42469,0.0,2011-03-24,Nethalan,583,RWE,37.339811,-7.065373,Ofisini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
7560,8437,0.0,2011-03-23,Nethalan,590,RWE,37.328905,-7.175174,Kwa Said Mlinda,0,...,never pay,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
8896,45838,0.0,2011-03-17,Nethalan,519,RWE,37.281357,-7.095131,Kwa Batolome,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
10267,49698,0.0,2011-03-17,Nethalan,583,RWE,37.339811,-7.065373,Ofisini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
13570,21056,0.0,2011-03-24,Tassaf,512,DWE,37.274352,-7.102004,Kwa Kinenge,0,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
15917,42947,1000.0,2011-03-17,Nethalan,621,RWE,37.375717,-7.056372,Kwa Yusta,0,...,monthly,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
17442,54643,0.0,2011-03-24,Nethalan,576,Hanja Lt,37.314250,-7.177203,Shule Yamsingi Maharaka,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
19866,3559,0.0,2011-03-24,Abood,486,Gwaseco,37.236326,-7.093914,Msikitini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
24938,33011,0.0,2011-03-17,Tassaf,532,DWE,37.297680,-7.078081,Kwa Gungulima,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
28701,58162,0.0,2011-03-24,Tassaf,532,DWE,37.297680,-7.078081,Kwa Gungulima,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


26 строк с повторяющимися данными. Отсортируем по возрастанию долготы:

In [159]:
TrainYEARSORT = TrainYEAR.loc[abs(TrainYEAR['longitude'] - 37.3) < 0.1]
TrainYEARSORT.sort_values(by = 'longitude')

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
29367,28980,0.0,2011-03-17,Abood,486,Gwaseco,37.236326,-7.093914,Msikitini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
19866,3559,0.0,2011-03-24,Abood,486,Gwaseco,37.236326,-7.093914,Msikitini,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
50330,36327,0.0,2011-03-24,Government Of Tanzania,499,DWE,37.250111,-7.104625,Kwa Magera,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
50232,44391,0.0,2011-03-17,Government Of Tanzania,499,DWE,37.250111,-7.104625,Kwa Magera,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
29640,8451,0.0,2011-03-17,Germany,496,RWE,37.252194,-7.103742,Kwa Magera,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,other,other
51319,26277,0.0,2011-03-24,Germany,496,RWE,37.252194,-7.103742,Kwa Magera,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,other,other
48361,31011,0.0,2011-03-24,Government Of Tanzania,503,DWE,37.269036,-7.104923,Kwa Beti,0,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
50878,12615,0.0,2011-03-17,Government Of Tanzania,503,DWE,37.269036,-7.104923,Kwa Beti,0,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
29872,60190,0.0,2011-03-17,Tassaf,512,DWE,37.274352,-7.102004,Kwa Kinenge,0,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
13570,21056,0.0,2011-03-24,Tassaf,512,DWE,37.274352,-7.102004,Kwa Kinenge,0,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [ ]:
Видим следующее: 
    1. У каждой из дублирующихся водокачек есть по 2 записи
    2. Даты первых и вторых записей у большинства из них совпадают
    3. Даты записей всех дублирующихся водокачек близки друг к другу
    
Делаем предположения:
    1. Сбором данных занимались две несообщённые группы - в таком случае можно исключить данные из выборки при условии
        неверности нижеследующего пункта.
    2. Записи сделаны по причине изменения состояния водокачек - проверяем.

In [ ]:
Извлечём из данных о состоянии водокачек две случайные записи под номерами, 
под которыми они дублируются в данных об их характеристиках:

In [181]:
#NewTrain = pd.merge(TrainYEARSORT, TrainingY, on = 'id','status_group', how = 'right')
#NewTrain

val = TrainYEARSORT
val1 = TrainingY
val1 = val1.merge(val, on = 'id', how = 'right')
val1.sort_values(by = 'latitude')

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
6,54643,functional,0.0,2011-03-24,Nethalan,576,Hanja Lt,37.314250,-7.177203,Shule Yamsingi Maharaka,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
14,18406,functional,0.0,2011-03-16,Nethalan,576,Hanja Lt,37.314250,-7.177203,Shule Yamsingi Maharaka,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
21,42578,functional,0.0,2011-03-23,Nethalan,580,DWE,37.318911,-7.177155,Kwa Kisuguru,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
24,14961,functional,0.0,2011-03-16,Nethalan,580,DWE,37.318911,-7.177155,Kwa Kisuguru,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
1,8437,non functional,0.0,2011-03-23,Nethalan,590,RWE,37.328905,-7.175174,Kwa Said Mlinda,...,never pay,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
15,64847,non functional,0.0,2011-03-16,Nethalan,590,RWE,37.328905,-7.175174,Kwa Said Mlinda,...,never pay,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
17,31011,non functional,0.0,2011-03-24,Government Of Tanzania,503,DWE,37.269036,-7.104923,Kwa Beti,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
22,12615,non functional,0.0,2011-03-17,Government Of Tanzania,503,DWE,37.269036,-7.104923,Kwa Beti,...,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
19,36327,non functional,0.0,2011-03-24,Government Of Tanzania,499,DWE,37.250111,-7.104625,Kwa Magera,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
18,44391,non functional,0.0,2011-03-17,Government Of Tanzania,499,DWE,37.250111,-7.104625,Kwa Magera,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other


Случайная проверка показала, что утверждение №2 неверно. Делаем вывод, что дублирующиеся данные скорее всего не дают никакой полезной информации, поэтому их можн (и даже нужно) удалить. Однако есть проблема: не во всех строчках заполнены все поля. Нам необходимо либо их дозаполнить, либо удалить записи по тем столбцам, которые заполнены у всех. 

## Дата ввода данных
Оценим разброс дат ввода данных. Если ввод был совершён слишком давно или слишком недавно относительно большинства дат, то такая дата может негативно сказаться на точности прогноза, т.к. если, например, запись была совершена за 40 